# Previous

In [1]:
import torch
from transformers import AutoTokenizer, AutoProcessor, AutoConfig
from qwen_vl_utils import process_vision_info

from src.qwen2_5.model import Qwen2_5_VLForConditionalGenerationWithHeatmap


path = "qwen-checkpoints"

processor = AutoProcessor.from_pretrained(path)
# config = AutoConfig.from_pretrained(path, trust_remote_code=True)
# config.vision_config.latent_dim = 256

model = Qwen2_5_VLForConditionalGenerationWithHeatmap.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    # config=config,
    attn_implementation="flash_attention_2",
    device_map="auto",
    trust_remote_code=True
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import Dataset

set_dataset_off = Dataset.load_from_disk("dried_heatmaps")

In [3]:
from src.common.dataset import DataCollator


data_collator = DataCollator(processor).data_collator

In [4]:
items = data_collator(set_dataset_off.select(range(10)))

In [9]:
model = model.eval()

In [10]:
for key in items:
    items[key] = items[key].cuda()

In [12]:
def gen_outputs(inputs):
    generated_ids = model.generate(**inputs, max_new_tokens=128)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, 
        # skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_text)
    return generated_ids, generated_ids_trimmed

gen_outputs(items)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


['<|endoftext|><|im_start|>\n<|im_start|>user<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_

(tensor([[151643, 151643, 151643,  ..., 151645, 151645, 151645],
         [151643, 151643, 151643,  ..., 151645, 151645, 151645],
         [151643, 151643, 151643,  ...,    198, 151644,    198],
         ...,
         [151643, 151643, 151643,  ..., 151645, 151645, 151645],
         [151643, 151643, 151643,  ..., 151645, 151645, 151645],
         [151643, 151643, 151643,  ...,    198, 151644,    198]],
        device='cuda:0'),
 [tensor([151643, 151644,    198, 151644,    872, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645,
          151645, 151645, 151645, 151645, 151645, 151645, 151645, 151645, 151

In [2]:
import torch
from transformers import AutoTokenizer, AutoProcessor, AutoConfig
from qwen_vl_utils import process_vision_info

from src.qwen2_5.model import Qwen2_5_VLForConditionalGenerationWithHeatmap


path = "Archistrax/Qwen2_5_VL"

processor = AutoProcessor.from_pretrained(path)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

class ResizeTensor:
    """
    Простой класс-трансформ, который повторяет поведение F.interpolate(..., align_corners=False)
    для тензора формата (C, H, W).
    """
    def __init__(self, size=(14, 14), mode='bilinear', align_corners=False):
        self.size = size
        self.mode = mode
        self.align_corners = align_corners

    def __call__(self, tensor):
        tensor = tensor.unsqueeze(0)  # -> (1, C, H, W)
        tensor = F.interpolate(
            tensor, size=self.size,
            mode=self.mode,
            align_corners=self.align_corners
        )  # -> (1, C, newH, newW)
        return tensor.squeeze(0)      # -> (C, newH, newW)

def get_heatmap_transformation(h, w):
    return transforms.Compose([
        transforms.Lambda(lambda img: img.convert("L")),
        transforms.ToTensor(),  # теперь тензор формата (1, H, W)
        transforms.Lambda(lambda t: (t - t.mean()) / (t.std() + 1e-8)),
        ResizeTensor(size=(int(h), int(w)), mode='bilinear', align_corners=False),
        transforms.Lambda(lambda t: t.flatten())
    ])



In [3]:
from typing import List, Dict, Any


image_folder = "./images/" 
messages_template = lambda image, transcription: [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_folder + image,
                "resized_height": 14*40,
                "resized_width": 14*80,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }, 
    {
        "role": "assistant",
        "content": [
            {"type": "text", "text": transcription},
        ],
    }
]

In [4]:
from typing import List


def find_substring(input_ids:torch.Tensor, ref_ids: List[int]):
    start_index = -1
    for i in range(len(input_ids) - len(ref_ids) + 1):
        if input_ids[i : i + len(ref_ids)].tolist() == ref_ids:
            start_index = i
            break
    if start_index == -1:
        raise ValueError("Target sequence not found.")
    end_index = start_index + len(ref_ids)
    return start_index, end_index


def create_labels(input_ids: torch.Tensor, answers: List[str]) -> torch.Tensor:
    """
    Create labels for SFT training. It masks all tokens after the start token with excluding_probability
    and after end token for the rest.
    Args:
        input_ids: ids from tokenizer output
        start_token: token that indicates start of selected sequence (for example, simple talk)
        end_token: token that indicates end of selected sequence
        excluding_probability: probability of excluding simple talk from attention

    Returns: tensor with masks  for each input_ids
    """

    labels = torch.full_like(input_ids, fill_value=-100)
    
    for i, row in enumerate(input_ids):
        start_index, end_index = find_substring(row, processor.tokenizer(answers[i], add_special_tokens=False)["input_ids"])
        labels[i, start_index:end_index] = row[start_index:end_index]

    return labels

In [5]:
answer_template = "<|im_start|>assistant\n{ans_text}<|im_end|>\n"

def process_injection(image_grid_thw, features):
    heatmap_flat = []
    for thw, feature in zip(image_grid_thw, features):
        _, h, w = thw
        transformation = get_heatmap_transformation(h/2, w/2)
        heatmap_flat.append(transformation(feature["heatmap"]).unsqueeze(1))

    return torch.stack(heatmap_flat)


def data_collator(features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
    if not features:
        return {}

    messages = []
    answers = []
    for feature in features:
        messages.append(messages_template(feature["image"], feature["transcribation"]))
        answers.append(answer_template.format(ans_text=feature["transcribation"]))

    texts = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    image_inputs, _ = process_vision_info(messages)
    batch = processor(text=texts, images=image_inputs, padding=True, return_tensors="pt")  # ['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw']

    batch["labels"] = create_labels(batch["input_ids"], answers)
    batch["heatmap_flat"] = process_injection(batch["image_grid_thw"], features)

    return batch
    

In [2]:
from datasets import Dataset

# for param in model.parameters():
#     param.requires_grad = False
# for param in model.visual.post_merger_injector.heatmap_proj.parameters():
#     param.requires_grad = True


set_dataset_off = Dataset.load_from_disk("dried_heatmaps")
# train_data = set_dataset_off.select(range(10))

In [5]:
set_dataset_off[0]["image"]

'abstraction_4.jpg'

In [7]:
from datasets import Dataset
from tqdm.auto import tqdm # Для индикатора прогресса

# Предположим, ваш датасет загружен в переменную `dataset`
# dataset = Dataset(...)
dataset = set_dataset_off

print(f"Исходный размер датасета: {len(dataset)}")

# --- Способ 1: Итерация с отслеживанием (Хорошо для умеренных размеров) ---

seen_combinations = set()
indices_to_keep = []

# Получаем доступ к нужным колонкам (это эффективно в datasets)
human_ids = dataset['human_id']
# ПРЕДПОЛОЖЕНИЕ: Значения в колонке 'image' являются сравнимыми и хэшируемыми
# (например, строки с путями к файлам или уникальные ID изображений).
# Если там сами объекты изображений (PIL/numpy), этот способ не сработает напрямую.
image_identifiers = dataset['image']

print("Поиск уникальных комбинаций (human_id, image)...")
for i in tqdm(range(len(dataset))):
    # Создаем кортеж для ключа (кортежи хэшируемы)
    combination = (human_ids[i], image_identifiers[i])

    if combination not in seen_combinations:
        seen_combinations.add(combination)
        indices_to_keep.append(i) # Сохраняем индекс первого встреченного уникального

# Создаем новый датасет, выбирая только нужные строки по индексам
unique_dataset = dataset.select(indices_to_keep)

print(f"Размер отфильтрованного датасета: {len(unique_dataset)}")

Исходный размер датасета: 16380
Поиск уникальных комбинаций (human_id, image)...


  0%|          | 0/16380 [00:00<?, ?it/s]

Размер отфильтрованного датасета: 833


In [9]:
unique_dataset.save_to_disk("dried_heatmaps")

Saving the dataset (0/1 shards):   0%|          | 0/833 [00:00<?, ? examples/s]

In [16]:
import torch
# from trl import SFTTrainer, Trainer
from transformers import TrainingArguments, TrainerCallback, Trainer
# from clearml import Task
from dotenv import load_dotenv

load_dotenv()

training_args = TrainingArguments(
    output_dir="./qwen-checkpoints",
    num_train_epochs=40,
    per_device_train_batch_size=2, #8?
    gradient_accumulation_steps=2,
    learning_rate=3e-4,
    weight_decay=0.01,
    lr_scheduler_type="cosine", #linear?
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    fp16=False,
    bf16=True,
    gradient_checkpointing=True,
    optim="adamw_8bit",
    push_to_hub=True,
    report_to=[],
    hub_model_id="Archistrax/Qwen2_5_VL-checkpoints",
    remove_unused_columns=False
)


# class ClearMLCallback(TrainerCallback):
#     def __init__(self, task):
#         self.task = task
#         self.logger = task.get_logger()

#     def on_log(self, args, state, control, logs=None, **kwargs):
#         if logs:
#             for key, value in logs.items():
#                 self.logger.report_scalar(title="Training", series=key, value=value, iteration=state.global_step)


# experiment_name = "qwen2.5-full"
# task = Task.init(
#     project_name="qwen2.5",
#     task_name=experiment_name,
#     output_uri=False
# )


tokenizer = processor.tokenizer
# clearml_callback = ClearMLCallback(task)
# logging.getLogger("clearml").setLevel(logging.CRITICAL)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    data_collator=data_collator,
    args=training_args,
    # callbacks=[clearml_callback]
)

/tmp/ipykernel_2076/1975460957.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,2.563000
20,2.612400
30,2.433500


KeyboardInterrupt: 

# Check

In [1]:
import torch
import os
from vlm_injector_train import HeatmapInjectionExperiment
from transformers import AutoProcessor, AutoConfig, PreTrainedModel, ProcessorMixin, TrainingArguments, Trainer


CONFIG_PATH = 'config/qwen2.5_heat.yaml'

if not os.path.exists(CONFIG_PATH):
    print(f"ОШИБКА: Конфигурационный файл не найден: {CONFIG_PATH}")
else:
    experiment = HeatmapInjectionExperiment(CONFIG_PATH)
    experiment.prepare_for_training()

    # Переменные для удобства
    model = experiment.model
    processor = experiment.processor
    tokenizer = processor.tokenizer
    train_dataset = experiment.train_dataset
    data_collator = experiment.data_collator
    cfg = experiment.cfg # Доступ к параметрам конфига

    trainer = Trainer(
        model=experiment.model,
        processing_class=experiment.processor,
        train_dataset=experiment.train_dataset,
        eval_dataset=experiment.eval_dataset,
        data_collator=experiment.data_collator,
        args=experiment.train_args,
        # callbacks=[ClearMLCallback(experiment.task)]
    )

# %% [markdown]
# ## 4. Настройка Устройства и DataLoader

# %%
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

# Перемещаем модель на устройство (если не используется device_map)
if not cfg.model.kwargs.get('device_map'):
     model.to(device)
     print(f"Модель перемещена на {device}")

# Создаем DataLoader для получения батча
# --- ВАЖНО: Установите batch_size как в вашем конфиге trainer! ---
batch_size = cfg.trainer.get('per_device_train_batch_size', 1) # Укажите ваш batch_size
print(f"Размер батча: {batch_size}")
train_dataloader = trainer.get_train_dataloader()

try:
    batch = next(iter(train_dataloader))
    print("Батч данных успешно получен.")
except Exception as e:
    print(f"Ошибка при получении батча: {e}")
    raise

try:
    batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
    print(f"Батч перемещен на {device}")
except Exception as e:
     print(f"Ошибка при перемещении батча на {device}: {e}")
     raise

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen2_5_VLForConditionalGenerationWithHeatmap were not initialized from the model checkpoint at ./Qwen2.5-VL-3B-Instruct and are newly initialized: ['visual.post_merger_injector.heatmap_proj.0.bias', 'visual.post_merger_injector.heatmap_proj.0.weight', 'visual.post_merger_injector.heatmap_proj.2.bias', 'visual.post_merger_injector.heatmap_proj.2.weight', 'visual.post_merger_injector.norm.bias', 'visual.post_merger_injector.norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Используемое устройство: cuda
Размер батча: 2
Батч данных успешно получен.
Батч перемещен на cuda


In [2]:
import torch
import torch.nn as nn
from torch import Tensor
from transformers.models.qwen2_5_vl.modeling_qwen2_5_vl import Qwen2_5_VisionTransformerPretrainedModel


class PostMergerFiLMInjector(nn.Module):
    def __init__(self, hidden_dim, intermediate_dim=None):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.norm = nn.RMSNorm(hidden_dim, eps=1e-4)

        intermediate_dim = hidden_dim // 4 if intermediate_dim is None else intermediate_dim
        self.heatmap_proj = nn.Sequential(
            nn.Linear(1, intermediate_dim),
            nn.GELU(),
            nn.Linear(intermediate_dim, 2 * hidden_dim)
        )

        nn.init.zeros_(self.heatmap_proj[-1].bias)
        nn.init.zeros_(self.heatmap_proj[-1].weight)

    def forward(self, hidden_states, heatmap_values):
        """
        hidden_states: (N, D)
        heatmap_values: (N, 1)
        """
        with torch.autocast(device_type=hidden_states.device.type, dtype=torch.float32):
            normed_hidden_states = self.norm(hidden_states.float())
        # normed_hidden_states = self.norm(hidden_states)  # (N, D)
        gamma_beta = self.heatmap_proj(heatmap_values)  # (N, 2*D)
        gamma, beta = torch.chunk(gamma_beta, 2, dim=-1)

        modulated_states = (1 + gamma) * normed_hidden_states + beta
        output_states = hidden_states + modulated_states

        return output_states


class PostMergerHeatmapInjector(nn.Module):
    def __init__(self, hidden_dim, latent_dim=None):
        super().__init__()
        latent_dim = latent_dim or hidden_dim // 4
        self.to_latent = nn.Sequential(
            nn.Linear(hidden_dim, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, latent_dim)
        )
        self.from_latent = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
        )
        self.heatmap_proj = nn.Sequential(
            nn.Linear(1, latent_dim),
            nn.ReLU(),
            nn.Linear(latent_dim, latent_dim),
        )

    def forward(self, hidden_states, heatmap_values):
        """
        hidden_states: (N, D)
        heatmap_values: (N, 1)
        """
        z = self.to_latent(hidden_states)  # (N, latent)
        h = self.heatmap_proj(heatmap_values)  # (N, latent)
        z_combined = z + h  # (N, latent)
        return self.from_latent(z_combined)  # (N, D)


class Qwen2_5_VisionTransformerWithHeatmap(Qwen2_5_VisionTransformerPretrainedModel):
    def __init__(self, config):
        super().__init__(config)
        # self.post_merger_injector = PostMergerHeatmapInjector(config.out_hidden_size, config.latent_dim)
        self.post_merger_injector = PostMergerFiLMInjector(config.out_hidden_size, config.latent_dim)

    def forward(self, hidden_states: Tensor, grid_thw: Tensor, heatmap_flat: Tensor = None) -> Tensor:
        hidden_states = super().forward(hidden_states, grid_thw)  # (N, D)
        if heatmap_flat is not None:
            print("--- Проверка hidden_states ПЕРЕД инжектором ---")
            is_bad_before = torch.isnan(hidden_states) | torch.isinf(hidden_states)
            if is_bad_before.any():
                print("!!! ОБНАРУЖЕНЫ NaN/Inf В hidden_states ПЕРЕД ИНЖЕКТОРОМ !!!")
            else:
                print(f"Статистика hidden_states перед инжектором: "
                      f"min={hidden_states.min().item():.4f}, "
                      f"max={hidden_states.max().item():.4f}, "
                      f"mean={hidden_states.mean().item():.4f}")
            # ---
            hidden_states = self.post_merger_injector(hidden_states, heatmap_flat)  # (N, D)
        return hidden_states


In [3]:
from typing import Optional, Tuple, List, Union

import torch

from torch.nn import CrossEntropyLoss
from transformers import Qwen2_5_VLConfig
from transformers.models.qwen2_5_vl.modeling_qwen2_5_vl import (
    Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLCausalLMOutputWithPast
)
from transformers.utils import logging

# from src.qwen2_5.vae.heatmap import Qwen2_5_VisionTransformerWithHeatmap

logger = logging.get_logger(__name__)


class Qwen2_5_VLForConditionalGenerationWithHeatmap(Qwen2_5_VLForConditionalGeneration):
    def __init__(self, config: Qwen2_5_VLConfig):
        super().__init__(config)
        self.visual = Qwen2_5_VisionTransformerWithHeatmap._from_config(config.vision_config)
        self.post_init()

    def forward(
            self,
            input_ids: torch.LongTensor = None,
            attention_mask: Optional[torch.Tensor] = None,
            position_ids: Optional[torch.LongTensor] = None,
            past_key_values: Optional[List[torch.FloatTensor]] = None,
            inputs_embeds: Optional[torch.FloatTensor] = None,
            labels: Optional[torch.LongTensor] = None,
            use_cache: Optional[bool] = None,
            output_attentions: Optional[bool] = None,
            output_hidden_states: Optional[bool] = None,
            return_dict: Optional[bool] = None,
            pixel_values: Optional[torch.Tensor] = None,
            pixel_values_videos: Optional[torch.FloatTensor] = None,
            image_grid_thw: Optional[torch.LongTensor] = None,
            video_grid_thw: Optional[torch.LongTensor] = None,
            rope_deltas: Optional[torch.LongTensor] = None,
            cache_position: Optional[torch.LongTensor] = None,
            second_per_grid_ts: Optional[torch.Tensor] = None,
            heatmap_flat=None,
    ) -> Union[Tuple, Qwen2_5_VLCausalLMOutputWithPast]:
        r"""
            labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
                Labels for computing the masked language modeling loss. Indices should either be in `[0, ...,
                config.vocab_size]` or -100 (see `input_ids` docstring). Tokens with indices set to `-100` are ignored
                (masked), the loss is only computed for the tokens with labels in `[0, ..., config.vocab_size]`.

        Returns:

        Example:

        ```python
        >>> from PIL import Image
        >>> import requests
        >>> from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration

        >>> model = Qwen2_5_VLForConditionalGeneration.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
        >>> processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

        >>> messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "What is shown in this image?"},
                ],
            },
        ]
        >>> url = "https://www.ilankelman.org/stopsigns/australia.jpg"
        >>> image = Image.open(requests.get(url, stream=True).raw)

        >>> text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        >>> inputs = processor(text=[text], images=[image], vision_infos=[vision_infos])

        >>> # Generate
        >>> generate_ids = model.generate(inputs.input_ids, max_length=30)
        >>> tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        "The image shows a street scene with a red stop sign in the foreground. In the background, there is a large red gate with Chinese characters ..."
        ```"""

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if inputs_embeds is None:
            inputs_embeds = self.model.embed_tokens(input_ids)
            if pixel_values is not None:
                pixel_values = pixel_values.type(self.visual.dtype)
                heatmap_flat = heatmap_flat.reshape(-1, 1).type(self.visual.dtype)  # (b*h*w, 1) Apply flattening for native injection as human attention
                image_embeds = self.visual(pixel_values, grid_thw=image_grid_thw, heatmap_flat=heatmap_flat)
                n_image_tokens = (input_ids == self.config.image_token_id).sum().item()
                n_image_features = image_embeds.shape[0]
                if n_image_tokens != n_image_features:
                    raise ValueError(
                        f"Image features and image tokens do not match: tokens: {n_image_tokens}, features {n_image_features}"
                    )

                mask = input_ids == self.config.image_token_id
                mask_unsqueezed = mask.unsqueeze(-1)
                mask_expanded = mask_unsqueezed.expand_as(inputs_embeds)
                image_mask = mask_expanded.to(inputs_embeds.device)

                image_embeds = image_embeds.to(inputs_embeds.device, inputs_embeds.dtype)
                inputs_embeds = inputs_embeds.masked_scatter(image_mask, image_embeds)

            if pixel_values_videos is not None:
                pixel_values_videos = pixel_values_videos.type(self.visual.dtype)
                video_embeds = self.visual(pixel_values_videos, grid_thw=video_grid_thw)
                n_video_tokens = (input_ids == self.config.video_token_id).sum().item()
                n_video_features = video_embeds.shape[0]
                if n_video_tokens != n_video_features:
                    raise ValueError(
                        f"Video features and video tokens do not match: tokens: {n_video_tokens}, features {n_video_features}"
                    )

                mask = input_ids == self.config.video_token_id
                mask_unsqueezed = mask.unsqueeze(-1)
                mask_expanded = mask_unsqueezed.expand_as(inputs_embeds)
                video_mask = mask_expanded.to(inputs_embeds.device)

                video_embeds = video_embeds.to(inputs_embeds.device, inputs_embeds.dtype)
                inputs_embeds = inputs_embeds.masked_scatter(video_mask, video_embeds)

            if attention_mask is not None:
                attention_mask = attention_mask.to(inputs_embeds.device)

        if position_ids is None and (attention_mask is None or attention_mask.ndim == 2):
            # calculate RoPE index once per generation in the pre-fill stage only
            if (
                    (cache_position is not None and cache_position[0] == 0)
                    or self.rope_deltas is None
                    or (past_key_values is None or past_key_values.get_seq_length() == 0)
            ):
                position_ids, rope_deltas = self.get_rope_index(
                    input_ids,
                    image_grid_thw,
                    video_grid_thw,
                    second_per_grid_ts,
                    attention_mask,
                )
                self.rope_deltas = rope_deltas
            # then use the prev pre-calculated rope-deltas to get the correct position ids
            else:
                batch_size, seq_length, _ = inputs_embeds.shape
                delta = (
                    (cache_position[0] + self.rope_deltas).to(inputs_embeds.device)
                    if cache_position is not None
                    else 0
                )
                position_ids = torch.arange(seq_length, device=inputs_embeds.device)
                position_ids = position_ids.view(1, -1).expand(batch_size, -1)
                if cache_position is not None:  # otherwise `deltas` is an int `0`
                    delta = delta.repeat_interleave(batch_size // delta.shape[0], dim=0)
                position_ids = position_ids.add(delta)
                position_ids = position_ids.unsqueeze(0).expand(3, -1, -1)

        outputs = self.model(
            input_ids=None,
            position_ids=position_ids,
            attention_mask=attention_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )

        hidden_states = outputs[0]
        logits = self.lm_head(hidden_states)

        loss = None
        if labels is not None:
            # Upcast to float if we need to compute the loss to avoid potential precision issues
            logits = logits.float()
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)

            # logits_to_check = shift_logits  # Тензор перед подачей в лосс
            #
            # if torch.isnan(logits_to_check).any():
            #     print("!!! ОБНАРУЖЕНЫ NaN В shift_logits ПЕРЕД ФУНКЦИЕЙ ПОТЕРЬ !!!")
            # elif torch.isinf(logits_to_check).any():
            #     print("!!! ОБНАРУЖЕНЫ Inf В shift_logits ПЕРЕД ФУНКЦИЕЙ ПОТЕРЬ !!!")
            # else:
            #     # Если нет NaN/Inf, проверим масштаб
            #     print(f"Статистика shift_logits перед лоссом: "
            #           f"min={logits_to_check.min().item():.4f}, "
            #           f"max={logits_to_check.max().item():.4f}, "
            #           f"mean={logits_to_check.mean().item():.4f}")
            #
            # # Также проверьте метки на всякий случай
            # if torch.any((shift_labels != -100) & ((shift_labels < 0) | (shift_labels >= self.config.vocab_size))):
            #     print("!!! ОБНАРУЖЕНЫ НЕКОРРЕКТНЫЕ ID В shift_labels (кроме -100) !!!")

            loss = loss_fct(shift_logits, shift_labels)

        if not return_dict:
            output = (logits,) + outputs[1:]
            return (loss,) + output if loss is not None else output

        return Qwen2_5_VLCausalLMOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
            rope_deltas=self.rope_deltas,
        )

In [4]:
# from src.qwen2_5.model import Qwen2_5_VLForConditionalGenerationWithHeatmap

name = "Qwen/Qwen2.5-VL-3B-Instruct"
hf_config = AutoConfig.from_pretrained(name, trust_remote_code=True)
hf_config.vision_config.latent_dim = 512

model = Qwen2_5_VLForConditionalGenerationWithHeatmap.from_pretrained(
    name,
    config=hf_config,
    # ignore_mismatched_sizes=True,
    **cfg.model.kwargs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Qwen2_5_VLForConditionalGenerationWithHeatmap were not initialized from the model checkpoint at Qwen/Qwen2.5-VL-3B-Instruct and are newly initialized: ['visual.post_merger_injector.heatmap_proj.0.bias', 'visual.post_merger_injector.heatmap_proj.0.weight', 'visual.post_merger_injector.heatmap_proj.2.bias', 'visual.post_merger_injector.heatmap_proj.2.weight', 'visual.post_merger_injector.norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
forward_hook_outputs = {}
backward_hook_grads = {}

# --- Функция для forward hook ---
def get_forward_hook(name):
    def hook(module, input, output):
        # Сохраняем выход (можно сохранять и input)
        # Клонируем и отсоединяем, чтобы не хранить весь граф вычислений
        if isinstance(output, torch.Tensor):
            data_to_store = output.detach().clone()
        elif isinstance(output, tuple): # Некоторые слои возвращают кортежи
            data_to_store = tuple(o.detach().clone() if isinstance(o, torch.Tensor) else o for o in output)
        else:
            data_to_store = output # Если не тензор, сохраняем как есть

        forward_hook_outputs[name] = data_to_store
        # Проверка на NaN/Inf на выходе
        if isinstance(data_to_store, torch.Tensor):
             if torch.isnan(data_to_store).any():
                 print(f"!!! ПОЙМАН NaN В FORWARD HOOK '{name}' (ВЫХОД) !!!")
             elif torch.isinf(data_to_store).any():
                 print(f"!!! ПОЙМАН Inf В FORWARD HOOK '{name}' (ВЫХОД) !!!")
        elif isinstance(data_to_store, tuple):
             for i, o in enumerate(data_to_store):
                 if isinstance(o, torch.Tensor):
                     if torch.isnan(o).any():
                         print(f"!!! ПОЙМАН NaN В FORWARD HOOK '{name}' (ВЫХОД, элемент {i}) !!!")
                     elif torch.isinf(o).any():
                         print(f"!!! ПОЙМАН Inf В FORWARD HOOK '{name}' (ВЫХОД, элемент {i}) !!!")

    return hook

# --- Функция для backward hook (регистрируем на ВЫХОД) ---
# register_full_backward_hook ловит grad_input и grad_output модуля
def get_backward_hook(name):
    def hook(module, grad_input, grad_output):
        # grad_output - градиент, пришедший К ЭТОМУ слою СВЕРХУ (ближе к лоссу)
        # grad_input - градиент, который уйдет ОТ ЭТОГО слоя ВНИЗ (к входам)
        data_to_store_in = tuple(g.detach().clone() if isinstance(g, torch.Tensor) else g for g in grad_input)
        data_to_store_out = tuple(g.detach().clone() if isinstance(g, torch.Tensor) else g for g in grad_output)
        backward_hook_grads[name] = {'grad_input': data_to_store_in, 'grad_output': data_to_store_out}

        # Проверяем grad_output (градиенты, пришедшие к слою)
        for i, g_out in enumerate(data_to_store_out):
             if isinstance(g_out, torch.Tensor):
                 if torch.isnan(g_out).any():
                     print(f"!!! ПОЙМАН NaN В BACKWARD HOOK '{name}' (GRAD_OUTPUT {i}) !!!")
                 elif torch.isinf(g_out).any():
                     print(f"!!! ПОЙМАН Inf В BACKWARD HOOK '{name}' (GRAD_OUTPUT {i}) !!!")

        # Проверяем grad_input (градиенты, уходящие от слоя)
        for i, g_in in enumerate(data_to_store_in):
             if isinstance(g_in, torch.Tensor):
                 if torch.isnan(g_in).any():
                     print(f"!!! ПОЙМАН NaN В BACKWARD HOOK '{name}' (GRAD_INPUT {i}) !!!")
                 elif torch.isinf(g_in).any():
                     print(f"!!! ПОЙМАН Inf В BACKWARD HOOK '{name}' (GRAD_INPUT {i}) !!!")

    return hook

# --- Регистрируем хуки на ключевые модули ---
hooks = []
target_modules = {
    "injector": model.visual.post_merger_injector,
    # Добавьте слои ВНУТРИ инжектора, если нужно детальнее
    "injector_norm": model.visual.post_merger_injector.norm,
    "injector_heatmap_proj": model.visual.post_merger_injector.heatmap_proj,
    # "injector_final_linear": model.visual.post_merger_injector.from_latent[-1], # Пример
    "final_logits_layer": model.lm_head # Слой, производящий финальные логиты
}

for name, module in target_modules.items():
    if module is not None:
        hooks.append(module.register_forward_hook(get_forward_hook(name)))
        hooks.append(module.register_full_backward_hook(get_backward_hook(name))) # Используем register_full_backward_hook
    else:
        print(f"Предупреждение: Модуль '{name}' не найден для установки хуков.")

print(f"Установлено {len(hooks)} хуков.")


Установлено 8 хуков.


In [6]:
for param in model.parameters():
    param.requires_grad = False
for param in model.visual.post_merger_injector.parameters():
    param.requires_grad = True

In [7]:
learning_rate = cfg.trainer.get('learning_rate', 1e-6)
optim_name = cfg.trainer.get('optim', 'adamw_8bit') # Используем adamw_torch для простоты, если 8bit вызывает проблемы

if optim_name == 'adamw_8bit' and 'bitsandbytes' in globals():
     try:
         import bitsandbytes.optim as bnb_optim
         optimizer = bnb_optim.AdamW8bit(model.parameters(), lr=learning_rate)
         print(f"Используется AdamW8bit с lr={learning_rate}")
     except ImportError:
         print("bitsandbytes не найден, использую torch.optim.AdamW")
         optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
else:
     optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
     print(f"Используется torch.optim.AdamW с lr={learning_rate}")


# Включаем детектор аномалий (замедляет, но дает лучший трейсбек для NaN в backward)
torch.autograd.set_detect_anomaly(True)

# Определяем контекст autocast в зависимости от конфига
dtype_context = torch.bfloat16 if cfg.trainer.get('bf16', False) else torch.float32
use_amp = (dtype_context != torch.float32)
print(f"Используется Autocast с dtype={dtype_context}?: {use_amp}")


Используется torch.optim.AdamW с lr=1e-05
Используется Autocast с dtype=torch.bfloat16?: True


In [8]:
print("\n--- Выполнение Forward Pass ---")
model.train() # Переводим модель в режим обучения
optimizer.zero_grad() # Обнуляем градиенты перед forward/backward

try:
    with torch.autocast(device_type=device.type, dtype=dtype_context, enabled=use_amp):
        # Передаем батч в модель. Модель должна вернуть словарь, включающий 'loss'
        outputs = model(**batch)
        # Извлекаем лосс
        if isinstance(outputs, dict) and 'loss' in outputs:
             loss = outputs['loss']
             print(f"Forward pass выполнен. Loss: {loss.item():.4f}")
        else:
             # Если модель не возвращает лосс, его нужно вычислить вручную
             # logits = outputs.logits # Или как они у вас называются
             # shift_logits = logits[..., :-1, :].contiguous()
             # shift_labels = batch['labels'][..., 1:].contiguous()
             # loss_fct = nn.CrossEntropyLoss()
             # loss = loss_fct(shift_logits.view(-1, model.config.vocab_size), shift_labels.view(-1))
             print("ПРЕДУПРЕЖДЕНИЕ: Модель не вернула 'loss'. Вычисление лосса не реализовано в этом скрипте отладки.")
             loss = None # Установим в None, чтобы пропустить backward

    # Проверка лосса на NaN/Inf
    if loss is not None:
         if torch.isnan(loss):
             print("!!! ПОЙМАН NaN В ИТОГОВОМ ЛОССЕ !!!")
         elif torch.isinf(loss):
             print("!!! ПОЙМАН Inf В ИТОГОВОМ ЛОССЕ !!!")

except Exception as e:
    print(f"ОШИБКА во время Forward Pass: {e}")
    loss = None # Не можем делать backward
    raise


--- Выполнение Forward Pass ---
--- Проверка hidden_states ПЕРЕД инжектором ---
Статистика hidden_states перед инжектором: min=-69.0000, max=32.5000, mean=0.0104
!!! ПОЙМАН NaN В FORWARD HOOK 'injector_norm' (ВЫХОД) !!!
!!! ПОЙМАН NaN В FORWARD HOOK 'injector' (ВЫХОД) !!!
!!! ПОЙМАН NaN В FORWARD HOOK 'final_logits_layer' (ВЫХОД) !!!
Forward pass выполнен. Loss: nan
!!! ПОЙМАН NaN В ИТОГОВОМ ЛОССЕ !!!


In [9]:

# --- Forward Pass ---


# --- Backward Pass ---
print("\n--- Выполнение Backward Pass ---")
if loss is not None and not torch.isnan(loss) and not torch.isinf(loss):
    try:
        # Без GradScaler для BF16 или FP32. Trainer использует accelerate.backward
        # Для простоты отладки используем прямой вызов backward()
        loss.backward()
        print("Backward pass выполнен.")

        # --- Проверка Градиентов Параметров ---
        print("\n--- Проверка Градиентов Обучаемых Параметров ---")
        nan_found_in_grads = False
        inf_found_in_grads = False
        zero_grad_count = 0
        non_zero_grad_count = 0

        # Проверяем градиенты только для обучаемых параметров инжектора
        injector_params = model.visual.post_merger_injector.parameters()
        for name, param in model.visual.post_merger_injector.named_parameters():
             if param.requires_grad:
                 if param.grad is None:
                     print(f"  Градиент для '{name}' равен None!")
                     zero_grad_count += 1
                 else:
                     if torch.isnan(param.grad).any():
                         print(f"!!! ПОЙМАН NaN В ГРАДИЕНТЕ '{name}' !!!")
                         nan_found_in_grads = True
                     elif torch.isinf(param.grad).any():
                         print(f"!!! ПОЙМАН Inf В ГРАДИЕНТЕ '{name}' !!!")
                         inf_found_in_grads = True
                     elif torch.count_nonzero(param.grad) == 0:
                          # print(f"  Градиент для '{name}' полностью нулевой.")
                          zero_grad_count += 1
                     else:
                          # print(f"  Градиент для '{name}' OK (ненулевой). Mean abs: {param.grad.abs().mean().item():.4g}")
                          non_zero_grad_count += 1
             # else: # Можно раскомментировать для проверки замороженных
             #     if param.grad is not None:
             #         print(f"ПРЕДУПРЕЖДЕНИЕ: Градиент НЕ None для замороженного параметра '{name}'")
        print(f"""Итог проверки градиентов: NaN={nan_found_in_grads}, Inf={inf_found_in_grads}, Нулевых/None={zero_grad_count}, Ненулевых={non_zero_grad_count}""")
    except RuntimeError as e:
        print(f"ОШИБКА во время Backward Pass: {e}")
        print("Вывод данных из хуков может помочь найти причину.")
    except Exception as e:
        print(f"Неожиданная ОШИБКА во время Backward Pass: {e}")
        raise



--- Выполнение Backward Pass ---


In [10]:
print("\n--- Выходные данные Forward Хуков ---")
for name, output_data in forward_hook_outputs.items():
    print(f"\nМодуль: {name}")
    if isinstance(output_data, torch.Tensor):
        print(f"  Shape: {output_data.shape}, Dtype: {output_data.dtype}, Device: {output_data.device}")
        if not torch.isnan(output_data).any() and not torch.isinf(output_data).any():
             print(f"  Stats: min={output_data.min().item():.4f}, max={output_data.max().item():.4f}, mean={output_data.mean().item():.4f}")
        else:
             print(f"  !!! Содержит NaN или Inf !!!")
    elif isinstance(output_data, tuple):
         print(f"  Кортеж с {len(output_data)} элементами:")
         for i, o in enumerate(output_data):
              if isinstance(o, torch.Tensor):
                  print(f"    Элемент {i}: Shape={o.shape}, Dtype={o.dtype}")
                  if not torch.isnan(o).any() and not torch.isinf(o).any():
                       print(f"      Stats: min={o.min().item():.4f}, max={o.max().item():.4f}, mean={o.mean().item():.4f}")
                  else:
                       print(f"      !!! Содержит NaN или Inf !!!")
              else:
                   print(f"    Элемент {i}: Тип={type(o)}")
    else:
        print(f"  Тип: {type(output_data)}")


print("\n--- Данные Backward Хуков ---")
for name, grad_data in backward_hook_grads.items():
    print(f"\nМодуль: {name}")
    print("  Grad Output (Градиенты, пришедшие к слою):")
    for i, g_out in enumerate(grad_data['grad_output']):
        if isinstance(g_out, torch.Tensor):
            print(f"    Элемент {i}: Shape={g_out.shape}, Dtype={g_out.dtype}")
            if not torch.isnan(g_out).any() and not torch.isinf(g_out).any():
                 print(f"      Stats: min={g_out.min().item():.4g}, max={g_out.max().item():.4g}, mean={g_out.mean().item():.4g}")
            else:
                 print(f"      !!! Содержит NaN или Inf !!!")
        else:
             print(f"    Элемент {i}: {type(g_out)}") # Часто None для входов, не требующих градиент

    print("  Grad Input (Градиенты, уходящие от слоя):")
    for i, g_in in enumerate(grad_data['grad_input']):
         if isinstance(g_in, torch.Tensor):
             print(f"    Элемент {i}: Shape={g_in.shape}, Dtype={g_in.dtype}")
             if not torch.isnan(g_in).any() and not torch.isinf(g_in).any():
                  print(f"      Stats: min={g_in.min().item():.4g}, max={g_in.max().item():.4g}, mean={g_in.mean().item():.4g}")
             else:
                  print(f"      !!! Содержит NaN или Inf !!!")
         else:
              print(f"    Элемент {i}: {type(g_in)}") # Может быть None для не-тензорных входов или входов, не требующих градиент



--- Выходные данные Forward Хуков ---

Модуль: injector_norm
  Shape: torch.Size([1600, 2048]), Dtype: torch.float32, Device: cuda:0
  !!! Содержит NaN или Inf !!!

Модуль: injector_heatmap_proj
  Shape: torch.Size([1600, 4096]), Dtype: torch.bfloat16, Device: cuda:0
  Stats: min=-0.2188, max=0.2246, mean=0.0000

Модуль: injector
  Shape: torch.Size([1600, 2048]), Dtype: torch.float32, Device: cuda:0
  !!! Содержит NaN или Inf !!!

Модуль: final_logits_layer
  Shape: torch.Size([2, 849, 151936]), Dtype: torch.bfloat16, Device: cuda:0
  !!! Содержит NaN или Inf !!!

--- Данные Backward Хуков ---


In [11]:
print("LayerNorm Weights:", model.visual.post_merger_injector.norm.weight.abs().mean().item())


LayerNorm Weights: nan


In [17]:
model.visual.post_merger_injector.norm.

RMSNorm((2048,), eps=0.0001, elementwise_affine=True)